## Dataset generation notebook
The following notebook is used to generate a dataset of trades and portofolio results to be analyzed later. 

### Dependencies

In [11]:
!pip install git+https://github.com/trsvchn/calabar.git
!pip3 install backtrader
!pip3 install pyfolio
!pip3 install tqdm
!pip3 install wheel
!pip3 install pandas
!pip3 install dask
!pip3 install graphviz
!pip3 install python-dateutil

  Cloning https://github.com/trsvchn/calabar.git to /tmp/pip-req-build-pakf3x57
  Running command git clone -q https://github.com/trsvchn/calabar.git /tmp/pip-req-build-pakf3x57
  Using cached empyrical-0.5.5-py3-none-any.whl


  Attempting uninstall: empyrical
    Found existing installation: empyrical 0.3.4
    Uninstalling empyrical-0.3.4:
      Successfully uninstalled empyrical-0.3.4
^C
ERROR: Operation cancelled by user


### Env variables and imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime
import os.path
import sys
import uuid
import copy
import json
import warnings

import pandas as pd
import requests
import tqdm
import backtrader as bt
import pyfolio as pf

warnings.filterwarnings("ignore")  # Avoid some noise

/home/narboom23/Projects/licenta/env/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
PROJECT_LOCATION = "/home/narboom23/Projects/licenta"
TICKERS_LOCATION = f"{PROJECT_LOCATION}/data/daily_tickers"
RESULTS_LOCATION = f"{PROJECT_LOCATION}/dataset/daily/results"

PARSED_TICKERS_JSON = f"{PROJECT_LOCATION}/dataset/daily/parsed_tickers.json"
TICKERS_JSON = f"{PROJECT_LOCATION}/dataset/daily/ticker_list.json"

## Tickers - uncomment the full json to use only a small sample
# Test sample
TICKER_NAMES = {
    "ATVI": "Activision Blizzard, Inc.",
    "EA": "Electronic Arts Inc.",
    "NTDOY": "Nintento (traded in US)",
    "GME": "GameStop Corp.",
    "GOOG": "Alphabet Inc.",
}

# Full json
with open(TICKERS_JSON, "r") as f:
    TICKER_NAMES = json.loads(f.read())


## Ticker list to be parsed
with open(PARSED_TICKERS_JSON, "r") as f:
    PARSED_TICKERS = json.loads(f.read())

ALL_TICKERS = list(TICKER_NAMES.keys())
PARSED_TICKER_NAMES = list(PARSED_TICKERS.keys())
TICKERS = [x for x in ALL_TICKERS if x not in PARSED_TICKER_NAMES]
NEXT_TICKER = TICKERS[0]
print(NEXT_TICKER)  # Useful for debugging stuck cronjobs


## Strategy defaults
DEFAULT_FROM_DATE = datetime.datetime(1998, 1, 1)
DEFAULT_TO_DATE = datetime.datetime(2020, 12, 31)
DEFAULT_CASH = 1000.0
DEFAULT_COMMISION = 0.0
DEFAULT_CPU_COUNT = 4


## Some other configs
KLASS_KEY = "klass"
RESULTS_FILENAME = f"{NEXT_TICKER}"

NUE


### Utilities

In [3]:
def add_entry_to_csv(csv_filename, trade_info_dict):
    csv_path = f"{RESULTS_LOCATION}/{csv_filename}.csv"

    trade_info_dict = {k: [v] for k, v in trade_info_dict.items()}

    trade_df = pd.DataFrame.from_dict(trade_info_dict)
    trade_df.to_csv(csv_path, mode="a", index=False, header=False)

In [4]:
def get_ticker_csv_path(ticker_name):
    return f"{TICKERS_LOCATION}/{ticker_name}.csv"


def get_ticker_csv_as_df(ticker_name):
    return pd.read_csv(get_ticker_csv_path(ticker_name))

In [5]:
def bt_opt_callback(cb):
    pbar.update()


def test_strategy(
    strategy_class,
    ticker_list,
    from_date=DEFAULT_FROM_DATE,
    to_date=DEFAULT_TO_DATE,
    cash=DEFAULT_CASH,
    commision=DEFAULT_COMMISION,
    cpu_count=DEFAULT_CPU_COUNT,
    **strategy_kwargs,
):

    # Create a cerebro entity
    cerebro = bt.Cerebro(optdatas=False)
    kwargs = {**strategy_kwargs}

    # Add a strategy
    strats = cerebro.optstrategy(strategy_class, **kwargs)

    datalist = [
        (get_ticker_csv_path(ticker_name), ticker_name) for ticker_name in ticker_list
    ]

    for i in range(len(ticker_list)):
        # Create a Data Feed
        data = bt.feeds.YahooFinanceCSVData(
            dataname=datalist[i][0],
            name=datalist[i][1],
            # Do not pass values before this date
            fromdate=from_date,
            # Do not pass values before this date
            todate=to_date,
            # Do not pass values after this date
            reverse=False,
        )

        # Add the Data Feed to Cerebro
        cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(cash)

    # Add pyfolio analyzer for stats
    #     cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(commission=commision)

    cerebro.optcallback(cb=bt_opt_callback)

    # Run over everything
    cerebro.run(maxcpus=cpu_count)

In [6]:
import time


def run_through_tickers(strategy_class, ticker_list, *args, **kwargs):
    valid_tickers = []
    for ticker_name in ticker_list:
        # Validate ticker
        with open(f"{TICKERS_LOCATION}/{ticker_name}.csv", "r") as f:
            if len(f.readlines()) > 550:
                valid_tickers.append(ticker_name)

    test_strategy(strategy_class, valid_tickers, *args, **kwargs)


def run_backtest_for_strategy_by_name(strategy_name, ticker_list=TICKERS):
    print(
        f"Running `run_backtest_for_strategy_by_name` for {strategy_name} for {len(ticker_list)} tickers"
    )
    strategy_setup = copy.deepcopy(STRATEGIES_ALL_CONFIGS)[strategy_name]
    strategy_klass = strategy_setup.pop(KLASS_KEY)
    return run_backtest_for_strategy(strategy_klass, ticker_list, strategy_setup)


def list_configs(configs):
    """Translate a configuration with ranges into a list of dicts with value pairs"""
    config_dict_list = [{"_dummy_param": 1}]

    for name, value in configs.items():
        if type(value) == range:
            new_config_dict_list = []
            for v in value:
                for cd in config_dict_list:
                    new_config_dict_list.append({name: v, **cd})
            config_dict_list = new_config_dict_list
        else:
            new_config_dict_list = []
            for cd in config_dict_list:
                new_config_dict_list.append({name: value, **cd})
            config_dict_list = new_config_dict_list

    for x in config_dict_list:
        x.pop("_dummy_param")

    return config_dict_list


def run_backtest_for_strategy(strategy_klass, ticker_list, configs):
    config_combination_list = list_configs(copy.deepcopy(configs))

    total_nr_jobs = len(ticker_list) * len(config_combination_list)

    run_through_tickers(strategy_klass, ticker_list, **configs)


def run_backtest_for_strategy_for_all_tickers(strategy_klass, configs):
    return run_backtest_for_strategy(strategy_klass, TICKERS, configs)

### Strategy
The main strategy class used

In [7]:
class RaynerTeoStrategy(bt.Strategy):
    """
    Rayner Teo Strategy with some additional logging

    Market:
      any stock

    Define the trend:
      (closing?) price above the 200-day moving average

    Entry:
      10-period RSI below 30 (buy on the next day's open)

    Exit:
      10-period RSI above 40, or after 10 trading days (sell on the next day's open)
    """

    params = (
        # TODO: these should be in a defaults class or something maybe
        # SMA
        ("maperiod", 15),
        # RSI
        ("rsi_open_period", 10),
        ("rsi_close_period", 30),
        # ADX
        ("adx_period", 14),
        # PPO
        ("ppo_period_short", 12),
        ("ppo_period_long", 26),
        # Stochastic
        ("stochastic_period", 14),
        # Other
        ("days_ago_close_period", 10),
        ("printlog", False),
        ("ticker", "GME"),
    )

    def log(self, txt, dt=None, doprint=False):
        """Logging function for this strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print("%s, %s" % (dt.isoformat(), txt))

    def __init__(self):
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()

            self.inds[d]["sma"] = bt.indicators.SimpleMovingAverage(
                d.close, period=self.params.maperiod
            )
            self.inds[d]["rsi"] = bt.indicators.RSI(
                d.close, period=self.params.rsi_open_period, safediv=True
            )

            self.inds[d]["adx8"] = bt.indicators.ADX(d, period=8)
            self.inds[d]["adx16"] = bt.indicators.ADX(d, period=16)
            self.inds[d]["adx32"] = bt.indicators.ADX(d, period=32)

            self.inds[d]["ppo8"] = bt.indicators.PPO(
                d.close, period1=8, period2=self.params.maperiod
            )  # , period_signal=?)
            self.inds[d]["ppo16"] = bt.indicators.PPO(
                d.close, period1=16, period2=self.params.maperiod
            )  # , period_signal=?)
            self.inds[d]["ppo32"] = bt.indicators.PPO(
                d.close, period1=32, period2=self.params.maperiod
            )  # , period_signal=?
            
            self.inds[d]["stochastic8"] = bt.indicators.Stochastic(
                d, period=8, safediv=True
            )
            self.inds[d]["stochastic16"] = bt.indicators.Stochastic(
                d, period=16, safediv=True
            )
            self.inds[d]["stochastic32"] = bt.indicators.Stochastic(
                d, period=32, safediv=True
            )

            self.inds[d]["order_placed_days_ago"] = 0

        self.csv_filename = RESULTS_FILENAME

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    "BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f"
                    % (order.executed.price, order.executed.value, order.executed.comm)
                )

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log(
                    "SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f"
                    % (order.executed.price, order.executed.value, order.executed.comm)
                )

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log("OPERATION PROFIT, GROSS %.2f, NET %.2f" % (trade.pnl, trade.pnlcomm))

    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size

            if not pos:  # no market / no orders
                if d.close[0] > self.inds[d]["sma"][0] and self.inds[d]["rsi"][0] < 30:
                    self.inds[d]["trade_info_dict"] = {
                        "uid": str(uuid.uuid1()),
                        "ticker": dn,
                        "date": self.data.datetime.date(),
                        "price_open": d.close[0],
                        "maperiod": self.params.maperiod,
                        "rsi_open_period": self.params.rsi_open_period,
                        "adx8": self.inds[d]["adx8"][0],
                        "adx16": self.inds[d]["adx16"][0],
                        "adx32": self.inds[d]["adx32"][0],
                        "ppo8": self.inds[d]["ppo8"][0],
                        "ppo16": self.inds[d]["ppo16"][0],
                        "ppo32": self.inds[d]["ppo32"][0],
                        "stochastic8": self.inds[d]["stochastic8"][0],
                        "stochastic16": self.inds[d]["stochastic16"][0],
                        "stochastic32": self.inds[d]["stochastic32"][0],
                    }
                    self.buy(data=d)
            else:
                if self.inds[d]["order_placed_days_ago"] == self.params.days_ago_close_period:
                    self.sell(data=d)

                    self.inds[d]["trade_info_dict"]["price_sell"] = d.close[0]
                    self.inds[d]["trade_info_dict"][
                        "days_ago_close_period"
                    ] = self.inds[d]["order_placed_days_ago"]
                    self.inds[d]["trade_info_dict"]["rsi_close_period"] = self.inds[d][
                        "rsi"
                    ][0]
                    
                    add_entry_to_csv(
                        csv_filename=self.csv_filename,
                        trade_info_dict=copy.deepcopy(self.inds[d]["trade_info_dict"]),
                    )

                    self.inds[d]["order_placed_days_ago"] = 0
                else:

                    self.inds[d]["trade_info_dict"]["price_sell"] = d.close[0]
                    self.inds[d]["trade_info_dict"][
                        "days_ago_close_period"
                    ] = self.inds[d]["order_placed_days_ago"]
                    self.inds[d]["trade_info_dict"]["rsi_close_period"] = self.inds[d][
                        "rsi"
                    ][0]
                    
                    add_entry_to_csv(
                        csv_filename=self.csv_filename,
                        trade_info_dict=copy.deepcopy(self.inds[d]["trade_info_dict"]),
                    )

                    self.inds[d]["order_placed_days_ago"] += 1

    def stop(self):
        self.log(
            f"(MA Period {self.params.maperiod}, "
            f"RSI open {self.params.rsi_open_period}, "
            f"RSI close {self.params.rsi_close_period}, "
            f"Close after {self.params.days_ago_close_period} days) "
            f"Ending Value {self.broker.getvalue()}",
            doprint=True,
        )

### Strategy run configs
Here are the configurations for particular strategy runs and the actual setup to run them

In [8]:
STRATEGIES_ALL_CONFIGS = {
    "Rayner Teo High Winrate": {
        KLASS_KEY: RaynerTeoStrategy,
        # Optimization
        "maperiod": range(180, 300, 20),
        "rsi_open_period": range(2, 16, 2),
        "days_ago_close_period": 20
        #         # Dummy run
        #         'maperiod': 256,
        #         'rsi_open_period': 8,
        #         'rsi_close_period': 32,
        #         'days_ago_close_period': 16
    },
}

In [9]:
from tqdm.auto import tqdm

strategy_name = "Rayner Teo High Winrate"

cfg_cpy = copy.deepcopy(STRATEGIES_ALL_CONFIGS[strategy_name])
cfg_cpy.pop(KLASS_KEY)
cfg_list = list_configs(cfg_cpy)
expected_number_of_tests = len(cfg_list)

pbar = tqdm(
    desc="Running backtests",
    leave=True,
    position=1,
    unit="run",
    colour="violet",
    total=expected_number_of_tests,
)
resulting_GME_df = run_backtest_for_strategy_by_name(
    "Rayner Teo High Winrate", ticker_list=[NEXT_TICKER]
)

PARSED_TICKERS[NEXT_TICKER] = TICKER_NAMES[NEXT_TICKER]

with open(PARSED_TICKERS_JSON, "w") as f:
    f.write(json.dumps(PARSED_TICKERS))

Running backtests:   0%|          | 0/42 [00:00<?, ?run/s]

Running `run_backtest_for_strategy_by_name` for Rayner Teo High Winrate for 1 tickers
2020-12-30, (MA Period 180, RSI open 8, RSI close 30, Close after 20 days) Ending Value 1318.1000000000008
2020-12-30, (MA Period 180, RSI open 6, RSI close 30, Close after 20 days) Ending Value 1237.699999999998
2020-12-30, (MA Period 180, RSI open 10, RSI close 30, Close after 20 days) Ending Value 1104.3000000000004
2020-12-30, (MA Period 180, RSI open 4, RSI close 30, Close after 20 days) Ending Value 1109.1999999999985
2020-12-30, (MA Period 180, RSI open 12, RSI close 30, Close after 20 days) Ending Value 1093.9999999999998
2020-12-30, (MA Period 180, RSI open 14, RSI close 30, Close after 20 days) Ending Value 1025.3000000000002
2020-12-30, (MA Period 180, RSI open 2, RSI close 30, Close after 20 days) Ending Value 1199.3999999999987
2020-12-30, (MA Period 200, RSI open 8, RSI close 30, Close after 20 days) Ending Value 1342.900000000001
2020-12-30, (MA Period 200, RSI open 6, RSI close 30, Clo